# TEEHR Evaluation Example
#####

## Part 3 - Evaluate Model Output
The prior notebooks were all about setting the stage for evaluation - preparing and joining the data to make the evaluation as easy and efficient as possible.  The reality is that evaluation is not remotely easy, particularly when you are dealing with very large datasets, 1000s of locations, many different models to compare, different baselines, different types of variables, different decision objectives, etc. There are a nearly endless number of ways to slice up the data, calculate metrics and visualize comparisons.  It is difficult (or impossible) to know in advance which approach is going to give us the most useful insights to answer a given question (e.g., which model is better for certain conditions and objectives?  how does it compare to a particular baselines? is there a relationship between performance and location characteristics (attributes)?

This notebook we will demonstrate how to use TEEHR to calculate metrics from the joined TEEHR database created in Notebook ex2-1, using a range of different options for grouping and filtering.  We will then create some common graphics based on the results.


#### In this notebook we will perform the following steps:
<ol>
    <li> Review the contents of our joined parquet file </li>
    <li> Calculate metrics with different group_by options </li>
    <li> Calculate metrics with different filters options </li>
    <li> Example visualizations of TEEHR results</li> 
</ol>

#### First setup the TEEHR class and review the contents of the joined parquet file

In [27]:
from teehr.classes.duckdb_joined_parquet import DuckDBJoinedParquet
from pathlib import Path

# Define the paths to the joined parquet file and the geometry files
TEEHR_BASE = Path(Path.home(), 'teehr/example-2')
JOINED_FILEPATH = f"{TEEHR_BASE}/joined/teehr_joined.parquet"
GEOMETRY_FILEPATH = f"{TEEHR_BASE}/geometry/**/*.parquet"

# Initialize a teehr joined parquet class with our parquet file and geometry
joined_data = DuckDBJoinedParquet(
    joined_parquet_filepath = JOINED_FILEPATH,
    geometry_filepath = GEOMETRY_FILEPATH
)

### 1. Review the contents of the joined parquet file

In practice, you may want to review the fields of data in the parquet file to plan your evaluation strategy.  If the dataset is large, reading it into a dataframe may be cumbersome or infeasible. TEEHR provides the ```get_joined_timeseries_schema``` method to quickly review the fields of the joined parquet file and the ```get_unique_field_values``` method to review the unique values contained in a specified field.  The latter is particularly helpful for building dashboards for evaluation (e.g., to populate a drop down menu of possible filter or group_by values).

In [28]:
joined_data.get_joined_timeseries_schema()

,column_name,column_type,null,key,default,extra
0,reference_time,TIMESTAMP,YES,None,None,None
1,value_time,TIMESTAMP,YES,None,None,None
2,secondary_location_id,VARCHAR,YES,None,None,None
3,secondary_value,FLOAT,YES,None,None,None
4,configuration,VARCHAR,YES,None,None,None
5,measurement_unit,VARCHAR,YES,None,None,None
6,variable_name,VARCHAR,YES,None,None,None
7,primary_value,FLOAT,YES,None,None,None
8,primary_location_id,VARCHAR,YES,None,None,None
9,q95_cms,VARCHAR,YES,None,None,None


In [29]:
# Review what configuration datasets were included
joined_data.get_unique_field_values('configuration')

,unique_configuration_values
0,marrmot_hbv
1,nwm30_retro


In [30]:
# ...number of locations
len(joined_data.get_unique_field_values('primary_location_id'))

543

In [31]:
# ... what variables were included
joined_data.get_unique_field_values('variable_name')

,unique_variable_name_values
0,streamflow_daily_mean


In [32]:
# ... and what unique attribute values are available for grouping
joined_data.get_unique_field_values('obs_flow_category_q_mean')

,unique_obs_flow_category_q_mean_values
0,high
1,low


In [33]:
joined_data.get_unique_field_values('river_forecast_center')

,unique_river_forecast_center_values
0,ABRFC
1,CBRFC
2,CNRFC
3,LMRFC
4,MARFC
5,MBRFC
6,NCRFC
7,NERFC
8,NWRFC
9,OHRFC


### 2. Calculate metrics with different group_by options

The ```get_metrics``` method for the joined parquet class works the same way as the 'on-the-fly' get_metrics function we ran in Notebook 2, ***except*** you do not need to specify any filepaths because those were already defined when creating the pre-joined parquet file and when initializing the joined parquet class above.

The arguments for the joined parquet class ```get_metrics``` are:

<ul>
    <li> group_by -> list of the fields TEEHR should use to group (or 'pool') the data for metric calculations (most often primary_location_id and configuration)* </li>
    <li> order_by -> list of fields on which the resulting table (dataframe) of metrics should be sorted</li>
    <li> include_metrics -> list of the available metrics to include in the results (available list link here) </li> 
    <li> filters (optional) ->  list of filters to extract a subset of the available data (e.g., locations, dates or value ranges)
    <li> include_geometry (optional) -> whether or not (True or False) the location geometry should be included in the results dataframe</li> 
</ul>

#### First, get the metrics for each location and configuration, for all data points available (no filters)

This is the same query we ran in Notebook 2 using the on-the-fly method.  Note the run time difference (~10s using on-the-fly vs. <1 s using pre-joined).  This might not seem significant, but when you scale up (hourly data, many model scenarios, 1000s of locations, etc.) the difference becomes much more significant. The pre-joined method will allow you to actually explore the data without waiting minutes (or hours) to recalculate metrics in different ways.

In [34]:
%%time

gdf_all = joined_data.get_metrics(
    group_by=["primary_location_id", "configuration"],
    order_by=["primary_location_id", "configuration"],
    include_metrics=[
        'kling_gupta_efficiency_mod2',
        'relative_bias',
        'pearson_correlation',                  
        'nash_sutcliffe_efficiency_normalized',  
        'mean_absolute_relative_error',
        'primary_count' 
    ],
    include_geometry=True,
)
# view the dataframe
gdf_all

CPU times: user 3.2 s, sys: 34.9 ms, total: 3.24 s
Wall time: 919 ms


,primary_location_id,configuration,kling_gupta_efficiency_mod2,relative_bias,pearson_correlation,nash_sutcliffe_efficiency_normalized,mean_absolute_relative_error,primary_count,geometry
0,usgs-01013500,marrmot_hbv,0.810593,0.083892,0.827273,0.740054,0.417619,4821,POINT (-68.58264 47.23739)
1,usgs-01013500,nwm30_retro,0.427555,-0.055877,0.844179,0.692448,0.474995,4821,POINT (-68.58264 47.23739)
2,usgs-01022500,marrmot_hbv,0.727231,-0.274713,0.874448,0.769694,0.434536,5549,POINT (-67.93524 44.60797)
3,usgs-01022500,nwm30_retro,0.684824,-0.184987,0.840962,0.756286,0.336071,5549,POINT (-67.93524 44.60797)
4,usgs-01030500,marrmot_hbv,0.918463,0.043953,0.929829,0.877706,0.297519,4879,POINT (-68.30596 45.50097)
...,...,...,...,...,...,...,...,...,...
1080,usgs-14303200,nwm30_retro,0.407590,0.357590,0.855616,0.564924,0.499118,7346,POINT (-123.54650 45.32428)
1081,usgs-14305500,marrmot_hbv,0.562337,-0.087626,0.688650,0.653981,0.530643,7596,POINT (-123.88733 44.71512)
1082,usgs-14305500,nwm30_retro,0.849484,-0.106285,0.866825,0.784714,0.422576,7596,POINT (-123.88733 44.71512)
1083,usgs-14316700,marrmot_hbv,0.750959,-0.006256,0.764942,0.695292,0.500118,7442,POINT (-122.72894 43.34984)


In [35]:
# use pandas magic to create a nice summary table of the metrics by model configuration across locations
gdf_all.groupby('configuration').describe(percentiles=[.5]).unstack(1).reset_index().rename(
    columns={'level_0':'metric','level_1':'summary'}).pivot(
    index=['metric','configuration'], values=0, columns='summary')

summary                                                     50%  count  \
metric                               configuration                       
kling_gupta_efficiency_mod2          marrmot_hbv       0.667960  542.0   
                                     nwm30_retro       0.655689  542.0   
mean_absolute_relative_error         marrmot_hbv       0.641694  542.0   
                                     nwm30_retro       0.517270  543.0   
nash_sutcliffe_efficiency_normalized marrmot_hbv       0.637779  542.0   
                                     nwm30_retro       0.691375  543.0   
pearson_correlation                  marrmot_hbv       0.710240  542.0   
                                     nwm30_retro       0.798239  542.0   
primary_count                        marrmot_hbv    6250.000000  542.0   
                                     nwm30_retro    6249.000000  543.0   
relative_bias                        marrmot_hbv       0.046592  542.0   
                                     nwm30_retro      -0.098476  543.0   

summary                                                     max         mean  \
metric                               configuration                             
kling_gupta_efficiency_mod2          marrmot_hbv       0.918463     0.637522   
                                     nwm30_retro       0.947258     0.577859   
mean_absolute_relative_error         marrmot_hbv       2.142410     0.704147   
                                     nwm30_retro       3.424756     0.576644   
nash_sutcliffe_efficiency_normalized marrmot_hbv       0.877706     0.633717   
                                     nwm30_retro       0.925337     0.665781   
pearson_correlation                  marrmot_hbv       0.929829     0.689824   
                                     nwm30_retro       0.959122     0.753647   
primary_count                        marrmot_hbv    7649.000000  5943.952030   
                                     nwm30_retro    7649.000000  5937.749540   
relative_bias                        marrmot_hbv       1.166685     0.055264   
                                     nwm30_retro       2.780231    -0.095718   

summary                                                    min          std  
metric                               configuration                           
kling_gupta_efficiency_mod2          marrmot_hbv     -0.168598     0.158162  
                                     nwm30_retro     -1.638418     0.298594  
mean_absolute_relative_error         marrmot_hbv      0.247937     0.263386  
                                     nwm30_retro      0.228409     0.276198  
nash_sutcliffe_efficiency_normalized marrmot_hbv      0.294141     0.088949  
                                     nwm30_retro      0.102859     0.120837  
pearson_correlation                  marrmot_hbv      0.001938     0.124738  
                                     nwm30_retro      0.010626     0.147716  
primary_count                        marrmot_hbv    743.000000  1346.167673  
                                     nwm30_retro    743.000000  1352.669068  
relative_bias                        marrmot_hbv     -0.565628     0.154332  
                                     nwm30_retro     -1.000000     0.346333

In [36]:
%%time

'''
Calculate metrics separately for low flows and high flows based on the 
calculated field "obs_flow_category_q_mean" -> add the field to the group_by list.  
'''

gdf_flowcat = joined_data.get_metrics(
    group_by=["primary_location_id", "configuration", "obs_flow_category_q_mean"],
    order_by=["primary_location_id", "configuration"],
    include_metrics=[
        'kling_gupta_efficiency_mod2',
        'pearson_correlation',                  
        'mean_absolute_relative_error',
        'primary_count' 
    ],
)
display(gdf_flowcat)
gdf_flowcat.groupby(['configuration','obs_flow_category_q_mean']).describe(percentiles=[.5]).unstack().unstack().reset_index().rename(
    columns={'level_0':'metric','level_1':'summary'}).pivot(
    index=['metric','obs_flow_category_q_mean','configuration'], values=0, columns='summary')


,primary_location_id,configuration,obs_flow_category_q_mean,kling_gupta_efficiency_mod2,pearson_correlation,mean_absolute_relative_error,primary_count
0,usgs-01013500,marrmot_hbv,low,-0.212347,0.519494,0.715540,2865
1,usgs-01013500,marrmot_hbv,high,0.698755,0.744577,0.333386,1956
2,usgs-01013500,nwm30_retro,low,-0.543014,0.658670,0.860122,2865
3,usgs-01013500,nwm30_retro,high,0.210697,0.741110,0.366104,1956
4,usgs-01022500,marrmot_hbv,low,0.071730,0.590457,0.674353,3464
...,...,...,...,...,...,...,...
2165,usgs-14305500,nwm30_retro,high,0.761461,0.802896,0.375993,2311
2166,usgs-14316700,marrmot_hbv,high,0.527434,0.589453,0.422769,2354
2167,usgs-14316700,marrmot_hbv,low,0.098754,0.652925,0.812883,5088
2168,usgs-14316700,nwm30_retro,high,0.741289,0.779103,0.411923,2354


CPU times: user 1.48 s, sys: 39.4 ms, total: 1.52 s
Wall time: 439 ms


summary                                                                      50%  \
metric                       obs_flow_category_q_mean configuration                
kling_gupta_efficiency_mod2  high                     marrmot_hbv       0.518226   
                                                      nwm30_retro       0.558464   
                             low                      marrmot_hbv      -0.823368   
                                                      nwm30_retro      -0.000192   
mean_absolute_relative_error high                     marrmot_hbv       0.531555   
                                                      nwm30_retro       0.478749   
                             low                      marrmot_hbv       0.972816   
                                                      nwm30_retro       0.624960   
pearson_correlation          high                     marrmot_hbv       0.587876   
                                                      nwm30_retro       0.727070   
                             low                      marrmot_hbv       0.439748   
                                                      nwm30_retro       0.571221   
primary_count                high                     marrmot_hbv    1610.500000   
                                                      nwm30_retro    1609.000000   
                             low                      marrmot_hbv    4553.500000   
                                                      nwm30_retro    4549.000000   

summary                                                              count  \
metric                       obs_flow_category_q_mean configuration          
kling_gupta_efficiency_mod2  high                     marrmot_hbv    542.0   
                                                      nwm30_retro    542.0   
                             low                      marrmot_hbv    542.0   
                                                      nwm30_retro    542.0   
mean_absolute_relative_error high                     marrmot_hbv    542.0   
                                                      nwm30_retro    543.0   
                             low                      marrmot_hbv    542.0   
                                                      nwm30_retro    543.0   
pearson_correlation          high                     marrmot_hbv    542.0   
                                                      nwm30_retro    542.0   
                             low                      marrmot_hbv    542.0   
                                                      nwm30_retro    542.0   
primary_count                high                     marrmot_hbv    542.0   
                                                      nwm30_retro    543.0   
                             low                      marrmot_hbv    542.0   
                                                      nwm30_retro    543.0   

summary                                                                      max  \
metric                       obs_flow_category_q_mean configuration                
kling_gupta_efficiency_mod2  high                     marrmot_hbv       0.852757   
                                                      nwm30_retro       0.893857   
                             low                      marrmot_hbv       0.561081   
                                                      nwm30_retro       0.788783   
mean_absolute_relative_error high                     marrmot_hbv       1.755141   
                                                      nwm30_retro       2.261760   
                             low                      marrmot_hbv     195.766492   
                                                      nwm30_retro     157.208326   
pearson_correlation          high                     marrmot_hbv       0.881226   
                                                      nwm30_retro       0.958468   
                             low                      marrmot_hbv       0.

In [37]:
%%time
'''
Now add the location characteristics you want included in the metrics table
(for output tables and visualization)

To include location-specific attributes in the metrics table, those attributes 
must be added to the group_by list.  If grouping across locations (.e.g., all locations 
within an RFC region), you should only add attributes that area already aggregated by that 
same region (TEEHR does not check for this). An example of including location characteristic 
attributes is included below. </li>

Notice we set include_geometry to False in this example so geometry is not included in the
resulting dataframe.

'''
# list the attributes that are location characteristics that you want to include 
# in the metrics results tables
include_location_characteristics = [
    'aridity',
    'runoff_ratio',
    'baseflow_index',
    'stream_order',  
    'q_mean_cms',
    'slope_fdc',  
    'frac_urban',
    'frac_snow',
    'forest_frac',
    'ecoregion_L2',
    'river_forecast_center',
]
df_atts = joined_data.get_metrics(
    group_by=["primary_location_id", "configuration"] + include_location_characteristics,
    order_by=["primary_location_id", "configuration"],
    include_metrics=[
        'kling_gupta_efficiency_mod2',
        'pearson_correlation',                  
        'mean_absolute_relative_error',
        'relative_bias',
        'primary_count' 
    ],
    include_geometry=False,
)

# view the dataframe
display(df_atts)

# summarize just the median results across locations by attribute (river forecast center)
df_atts_summary = df_atts.groupby(['configuration','river_forecast_center'])\
    .describe(percentiles=[.5]).unstack().unstack().reset_index()\
    .rename(columns={'level_0':'metric','level_1':'summary'})
df_atts_summary[df_atts_summary['summary'].isin(['50%'])].pivot(
    index=['river_forecast_center','configuration'],values=0, columns=['metric','summary'])

,primary_location_id,configuration,aridity,runoff_ratio,baseflow_index,stream_order,q_mean_cms,slope_fdc,frac_urban,frac_snow,forest_frac,ecoregion_L2,river_forecast_center,kling_gupta_efficiency_mod2,pearson_correlation,mean_absolute_relative_error,relative_bias,primary_count
0,usgs-01013500,marrmot_hbv,0.63055865946247,0.543437466590222,0.585225955779508,5,44.467109455834866,1.52821853538976,0.01,0.313440357191799,0.9063,8.1 MIXED WOOD PLAINS,NERFC,0.810593,0.827273,0.417619,0.083892,4821
1,usgs-01013500,nwm30_retro,0.63055865946247,0.543437466590222,0.585225955779508,5,44.467109455834866,1.52821853538976,0.01,0.313440357191799,0.9063,8.1 MIXED WOOD PLAINS,NERFC,0.427555,0.844179,0.474995,-0.055877,4821
2,usgs-01022500,marrmot_hbv,0.587356423405076,0.602268929482991,0.554478447930409,5,14.786380055715862,1.77627980351081,0.0092,0.245259009248271,0.9232,8.1 MIXED WOOD PLAINS,NERFC,0.727231,0.874448,0.434536,-0.274713,5549
3,usgs-01022500,nwm30_retro,0.587356423405076,0.602268929482991,0.554478447930409,5,14.786380055715862,1.77627980351081,0.0092,0.245259009248271,0.9232,8.1 MIXED WOOD PLAINS,NERFC,0.684824,0.840962,0.336071,-0.184987,5549
4,usgs-01030500,marrmot_hbv,0.624111385131731,0.555858982560286,0.508440712580478,5,77.36721025688733,1.87111040605632,0.0067,0.27701840295357,0.8782,8.1 MIXED WOOD PLAINS,NERFC,0.918463,0.929829,0.297519,0.043953,4879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,usgs-14303200,nwm30_retro,0.322802335415838,0.800308618058677,0.538191735950273,2,0.49607326587346307,2.00875019609279,0.0,0.088754104072909,1.0,7.1 MARINE WEST COAST FOREST,NWRFC,0.407590,0.855616,0.499118,0.357590,7346
1081,usgs-14305500,marrmot_hbv,0.325769239215451,0.99025270322957,0.520848289776854,5,39.99703741748896,1.94068087775056,0.004699999999999999,0.0257414277847911,0.9943,7.1 MARINE WEST COAST FOREST,NWRFC,0.562337,0.688650,0.530643,-0.087626,7596
1082,usgs-14305500,nwm30_retro,0.325769239215451,0.99025270322957,0.520848289776854,5,39.99703741748896,1.94068087775056,0.004699999999999999,0.0257414277847911,0.9943,7.1 MARINE WEST COAST FOREST,NWRFC,0.849484,0.866825,0.422576,-0.106285,7596
1083,usgs-14316700,marrmot_hbv,0.501305088892464,0.643997143591881,0.508616082222394,5,19.909239533259516,2.23102329485064,0.0,0.176336580742005,1.0,6.2 WESTERN CORDILLERA,NWRFC,0.750959,0.764942,0.500118,-0.006256,7442


CPU times: user 4.35 s, sys: 15 ms, total: 4.36 s
Wall time: 1.28 s


metric                              kling_gupta_efficiency_mod2  \
summary                                                     50%   
river_forecast_center configuration                               
ABRFC                 marrmot_hbv                      0.580993   
                      nwm30_retro                      0.604874   
CBRFC                 marrmot_hbv                      0.724193   
                      nwm30_retro                      0.591256   
CNRFC                 marrmot_hbv                      0.656739   
                      nwm30_retro                      0.673399   
LMRFC                 marrmot_hbv                      0.650767   
                      nwm30_retro                      0.780128   
MARFC                 marrmot_hbv                      0.703847   
                      nwm30_retro                      0.673464   
MBRFC                 marrmot_hbv                      0.587405   
                      nwm30_retro                      0.458590   
NCRFC                 marrmot_hbv                      0.643769   
                      nwm30_retro                      0.561194   
NERFC                 marrmot_hbv                      0.750251   
                      nwm30_retro                      0.691786   
NWRFC                 marrmot_hbv                      0.690916   
                      nwm30_retro                      0.793934   
OHRFC                 marrmot_hbv                      0.697471   
                      nwm30_retro                      0.744009   
SERFC                 marrmot_hbv                      0.695676   
                      nwm30_retro                      0.670965   
WGRFC                 marrmot_hbv                      0.531054   
                      nwm30_retro                      0.468307   

metric                              pearson_correlation  \
summary                                             50%   
river_forecast_center configuration                       
ABRFC                 marrmot_hbv              0.645221   
                      nwm30_retro              0.804665   
CBRFC                 marrmot_hbv              0.759280   
                      nwm30_retro              0.735411   
CNRFC                 marrmot_hbv              0.698656   
                      nwm30_retro              0.863914   
LMRFC                 marrmot_hbv              0.714142   
                      nwm30_retro              0.834349   
MARFC                 marrmot_hbv              0.725204   
                      nwm30_retro              0.800666   
MBRFC                 marrmot_hbv              0.665207   
                      nwm30_retro              0.706156   
NCRFC                 marrmot_hbv              0.684247   
                      nwm30_retro              0.751903   
NERFC                 marrmot_hbv              0.773721   
                      nwm30_retro              0.796547   
NWRFC                 marrmot_hbv              0.727940   
                      nwm30_retro              0.865328   
OHRFC                 marrmot_hbv              0.728472   
                      nwm30_retro              0.807639   
SERFC                 marrmot_hbv              0.718701   
                      nwm30_retro              0.790086   
WGRFC                 marrmot_hbv              0.617819   
                      nwm30_retro              0.688323   

metric                              mean_absolute_relative_error  \
summary                                                      50%   
river_forecast_center configuration                                
ABRFC                 marrmot_hbv                       0.953922   
                      nwm30_retro                       0.617173   
CBRFC                 marrmot_hbv                       0.721027   
                      nwm30_retro                       0.758276   
CNRFC                 marrmot_hbv                       0.727673   
                      nwm30_retro                   

### 3. Calculate metrics with different filters options

The filter option allows you to calculate metrics for desired subsets of the data.  Filters can be combined resulting in nearly endless possibilities, but some examples could include:

<ul>
    <li> A date range to isolate a calibration period and validation period</li>
    <li> Stream orders < 4 to isolate performance in small streams   </li>
    <li> RFC to get results only for a specific RFC region </li>
    <li> Aridity greater [less] than a threshold to get results only for arid [humid] locations </li>
    <li> Specified months to assess performance only for a specified season </li>
</ul>

The example below gets metrics results for small streams in the summer in the southeast and creates a KGE'' map of results.  Try different filters of your choice.


In [38]:
%%time

import geoviews as gv
import holoviews as hv
import colorcet as cc
hv.extension('bokeh', logo=False)
gv.extension('bokeh', logo=False)
basemap = hv.element.tiles.CartoLight()

gdf_filters = joined_data.get_metrics(
    group_by=["primary_location_id", "configuration", "stream_order"],
    order_by=["primary_location_id", "configuration"],
    include_metrics=[
        'kling_gupta_efficiency_mod2',
        'relative_bias',
        'pearson_correlation',                  
        'nash_sutcliffe_efficiency_normalized',  
        'mean_absolute_relative_error',
        'primary_count' 
    ],
    filters = [
          {
              "column": "stream_order",
              "operator": "in",
              "value": ['1','2','3','4']
              #"value": ['5','6','7','8']
          },
         # {
         #     "column": "month",
         #     "operator": "in",
         #     "value": ['5','6','7','8','9']
         # },
         # {
         #     "column": "river_forecast_center",
         #     "operator": "=",
         #     "value": "SERFC"
         # },
    ],
    include_geometry=True,
)
#display(gdf_filters.head())

# make a quick map of locations - see how it changes as you make different filter selections
basemap * gv.Points(gdf_filters, vdims=['kling_gupta_efficiency_mod2','configuration']).select(
    configuration='nwm30_retro').opts(
    color='kling_gupta_efficiency_mod2', 
    height=400, width=600, size=7, 
    cmap=cc.rainbow[::-1], colorbar=True, clim=(0,1))

CPU times: user 3.26 s, sys: 55.8 ms, total: 3.31 s
Wall time: 934 ms


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (kling_gupta_efficiency_mod2,configuration)

### 4. Example visualizations of TEEHR evaluation results

Just as there are many ways to calculate metrics, there are ***even more*** ways to visualize the results.  TEEHR does not currently include methods to generate plots, though that capability may be added in it becomes clear it would be useful.  Instead of building visualization functionality directly in TEEHR, our focus has been using one of the many available and powerful python packages for plotting in conjunction with TEEHR. The above map and below examples are all created using the holoviz suite of visualization tools (holoviz.org), with the 'bokeh' backend, which includes some interactive functionality by default. These examples are just barely scratching the surface of what's feasible with these tools. Holoviews makes it easy to create interactive dashboards to explore the vast range of performance relationships, patterns, and other insights that can be revealed by TEEHR evaluation results like those above.  This was too much to fit into the workshop, but we will share examples and future workshops will may explore more visualization methods. We would love feedback about which types of visualization approaches the community sees as top priorities to include in TEEHR directly.

In [39]:
# set up color and abbrevation settings to use across multiple plots

metric_abbrev=dict(
    kling_gupta_efficiency_mod2 = "KGE''",
    mean_absolute_relative_error = "MAE",
    pearson_correlation = "Correlation",
    relative_bias  = "Rel.Bias",
    nash_sutcliffe_efficiency_normalized = "NNSE",
)
cmap_lin = cc.rainbow[::-1]
cmap_div = cc.CET_D1A[::-1]
metric_colors=dict(
    kling_gupta_efficiency_mod2          = {'cmap': cmap_lin, 'clim': (0,1)},  
    relative_bias                        = {'cmap': cmap_div, 'clim': (-1,1)},   
    pearson_correlation                  = {'cmap': cmap_lin, 'clim': (0,1)},     
    nash_sutcliffe_efficiency_normalized = {'cmap': cmap_lin, 'clim': (0,1)}, 
    mean_absolute_relative_error         = {'cmap': cmap_lin, 'clim': (0,2)},
)
metrics = list(metric_colors.keys())
configs = ['nwm30_retro', 'marrmot_hbv']
config_colors = dict(marrmot_hbv='red', nwm30_retro='#377EB8')

#### 4a. Metric maps
First we will create side-by-side maps of the first query results above (all locations and configurations, no filters), showing metric values at each location, where dots are colored by metric value and sized by sample size.  See how the comparison changes for each metric.

In [41]:
# map_metric = 'kling_gupta_efficiency_mod2'
# map_metric = 'pearson_correlation'                  
# map_metric = 'nash_sutcliffe_efficiency_normalized'
# map_metric = 'mean_absolute_relative_error' 
map_metric = 'relative_bias'

# factor to size dots based on sample size 
size_factor = 15/max(gdf_filters[('primary_count')])

polys = gv.Points(
    gdf_all, 
    vdims = metrics + ['primary_location_id','configuration','primary_count'],
    label = 'metric value (color), sample size (size)',
).opts(
    height = 400,
    width = 600,
    line_color = 'gray',
    colorbar = True,
    size = hv.dim('primary_count') * 15/max(gdf_filters[('primary_count')]),
    tools = ['hover'],
    xaxis = 'bare',
    yaxis = 'bare',
    show_legend = True
)
maps = []
for config in configs:
    maps.append(basemap * polys.select(configuration=config).opts(
            title=f"{config} | {metric_abbrev[map_metric]}",
            color = map_metric,
            clim = metric_colors[map_metric]['clim'],
            cmap = metric_colors[map_metric]['cmap']
        )
    )
maps[0] + maps[1]

:Layout
   .Overlay.I  :Overlay
      .Tiles.I                                                                                                                :Tiles   [x,y]
      .Points.Metric_value_left_parenthesis_color_right_parenthesis_comma_sample_size_left_parenthesis_size_right_parenthesis :Points   [Longitude,Latitude]   (kling_gupta_efficiency_mod2,relative_bias,pearson_correlation,nash_sutcliffe_efficiency_normalized,mean_absolute_relative_error,primary_location_id,configuration,primary_count)
   .Overlay.II :Overlay
      .Tiles.I                                                                                                                :Tiles   [x,y]
      .Points.Metric_value_left_parenthesis_color_right_parenthesis_comma_sample_size_left_parenthesis_size_right_parenthesis :Points   [Longitude,Latitude]   (kling_gupta_efficiency_mod2,relative_bias,pearson_correlation,nash_sutcliffe_efficiency_normalized,mean_absolute_relative_error,primary_location_id,configuration,primary_count)

#### 4b. Dataframe table and bar chart side by side
Next we will summarize results across locations by creating a summary table with pandas (as we did above) and juxtapose it with a bar chart using holoviews and panel.

In [42]:
# Display dataframes and simple plots side by side using Panel
import panel as pn

gdf_summary = gdf_all.groupby('configuration').describe(percentiles=[.5]).unstack(1).reset_index().rename(
    columns={'level_0':'metric','level_1':'summary'}).pivot(
    index=['metric','configuration'], values=0, columns='summary')

gdf_bars = gdf_summary.drop('primary_count', axis=0)['50%'].reset_index().replace({'metric':metric_abbrev})
bars = hv.Bars(gdf_bars, kdims=['metric', 'configuration']).opts(
    xrotation=90, height=400, width=400, ylabel='median',xlabel='')

pn.Row(pn.pane.DataFrame(gdf_summary, width=800), bars)

Row
    [0] DataFrame(DataFrame, width=800)
    [1] HoloViews(Bars, height=400, sizing_mode='fixed', width=400)

#### 4c. Box-whisker plots of results by metric and model

Next we'll create box-whisker plots to see the distribution of metrics across locations for each metric and configuration.

In [43]:
# remove geometry so holoviews knows this is not a map.
df = gdf_all.drop('geometry', axis=1)

opts = dict(
    show_legend=False, 
    width=200, 
    cmap='Set1', 
    xrotation=45,
    labelled=[]
)
boxplots = []
for metric in metrics:
    boxplots.append(
        hv.BoxWhisker(df, 'configuration', metric, label=metric_abbrev[metric]).opts(
            **opts,
            box_fill_color=hv.dim('configuration')
        )
    )
hv.Layout(boxplots).cols(len(metrics))

:Layout
   .BoxWhisker.KGE_apostrophe_apostrophe :BoxWhisker   [configuration]   (kling_gupta_efficiency_mod2)
   .BoxWhisker.Rel_full_stop_Bias        :BoxWhisker   [configuration]   (relative_bias)
   .BoxWhisker.Correlation               :BoxWhisker   [configuration]   (pearson_correlation)
   .BoxWhisker.NNSE                      :BoxWhisker   [configuration]   (nash_sutcliffe_efficiency_normalized)
   .BoxWhisker.MAE                       :BoxWhisker   [configuration]   (mean_absolute_relative_error)

#### 4d. Histograms by metric and model
Every good scientist loves a histogram.  The below example creates a layout of histograms by configuration and metric, which gives us a more complete understanding of the metric distributions.

In [44]:
import hvplot.pandas
histograms =[]
for config in configs:
    for metric in metrics:
        histograms.append(
            df[df['configuration']==config].hvplot.hist(
                y=metric, 
                ylim=(0,200),
                color=config_colors[config],
                bin_range=metric_colors[metric]['clim'], 
                xlabel=metric_abbrev[metric],
            ).opts(height = 200, width=250, title = config)
        )
hv.Layout(histograms).cols(len(metrics))

:Layout
   .Histogram.I    :Histogram   [kling_gupta_efficiency_mod2]   (kling_gupta_efficiency_mod2_count)
   .Histogram.II   :Histogram   [relative_bias]   (relative_bias_count)
   .Histogram.III  :Histogram   [pearson_correlation]   (pearson_correlation_count)
   .Histogram.IV   :Histogram   [nash_sutcliffe_efficiency_normalized]   (nash_sutcliffe_efficiency_normalized_count)
   .Histogram.V    :Histogram   [mean_absolute_relative_error]   (mean_absolute_relative_error_count)
   .Histogram.VI   :Histogram   [kling_gupta_efficiency_mod2]   (kling_gupta_efficiency_mod2_count)
   .Histogram.VII  :Histogram   [relative_bias]   (relative_bias_count)
   .Histogram.VIII :Histogram   [pearson_correlation]   (pearson_correlation_count)
   .Histogram.IX   :Histogram   [nash_sutcliffe_efficiency_normalized]   (nash_sutcliffe_efficiency_normalized_count)
   .Histogram.X    :Histogram   [mean_absolute_relative_error]   (mean_absolute_relative_error_count)

#### 4e. CDFs overlays by metric
Every good scientist loves a CDF even more.  The below example creates a layout of histograms by configuration and metric, which gives us a more complete understanding of the metric distributions.  We include metrics here with (mostly) the same range (0,1) and 'good' value (1).  

In [45]:
import numpy as np

layout = []
for metric in [
    'kling_gupta_efficiency_mod2',
    'pearson_correlation',                  
    'nash_sutcliffe_efficiency_normalized',
]:
    xlim = metric_colors[metric]['clim']
    xlabel = metric_abbrev[metric]
    
    cdfs = hv.Curve([])
    for config in configs:
        data = df[df['configuration']==config]
        data[xlabel] = np.sort(data[metric])
        n = len(data[xlabel])
        data['y'] = 1. * np.arange(n) / (n - 1)    
        cdfs = cdfs * hv.Curve(data, xlabel, 'y', label=config).opts(color=config_colors[config])
        
    layout.append(
        cdfs.opts(
            width = 300,
            legend_position='top_left',
            xlim=xlim, 
            xlabel=xlabel,
            title=metric_abbrev[metric],
            shared_axes=False,
        )
    )
    
hv.Layout(layout).cols(5)

:Layout
   .Overlay.I   :Overlay
      .Curve.I           :Curve   [x]   (y)
      .Curve.Nwm30_retro :Curve   [KGE'']   (y)
      .Curve.Marrmot_hbv :Curve   [KGE'']   (y)
   .Overlay.II  :Overlay
      .Curve.I           :Curve   [x]   (y)
      .Curve.Nwm30_retro :Curve   [Correlation]   (y)
      .Curve.Marrmot_hbv :Curve   [Correlation]   (y)
   .Overlay.III :Overlay
      .Curve.I           :Curve   [x]   (y)
      .Curve.Nwm30_retro :Curve   [NNSE]   (y)
      .Curve.Marrmot_hbv :Curve   [NNSE]   (y)

#### 4f. Bar charts by attribute
In the third example query above, we demonstrate how to add attributes to the resulting dataframe for summary and visualization purposes.  In that example we generated a summary table to RFC region.  The below example uses those result to build bar charts of the median performance metric across locations within each RFC region.

In [46]:
df_bars = df_atts_summary.set_index('metric').drop('primary_count', axis=0).reset_index().set_index('summary').loc['50%']
df_bars = df_bars.replace({'metric': metric_abbrev}) \
    .rename(columns={'river_forecast_center':'rfc',0:'median'}) \
    .reset_index().drop('summary', axis=1)
df_bars.loc[df_bars['metric'] == 'MAE', 'median'] = 1 - df_bars.loc[df_bars['metric'] == 'MAE', 'median']
df_bars = df_bars.replace('MAE','1-MAE')

bars = hv.Bars(df_bars, kdims=['metric', 'configuration','rfc'],vdims=['median']).opts(
    xrotation=90, height=300, width=300, ylabel='median',xlabel='')

layout = []
for rfc in df_bars['rfc'].unique():
    layout.append(bars.select(rfc=rfc).opts(title=rfc))
hv.Layout(layout)

:Layout
   .Bars.I    :Bars   [metric,configuration,rfc]   (median)
   .Bars.II   :Bars   [metric,configuration,rfc]   (median)
   .Bars.III  :Bars   [metric,configuration,rfc]   (median)
   .Bars.IV   :Bars   [metric,configuration,rfc]   (median)
   .Bars.V    :Bars   [metric,configuration,rfc]   (median)
   .Bars.VI   :Bars   [metric,configuration,rfc]   (median)
   .Bars.VII  :Bars   [metric,configuration,rfc]   (median)
   .Bars.VIII :Bars   [metric,configuration,rfc]   (median)
   .Bars.IX   :Bars   [metric,configuration,rfc]   (median)
   .Bars.X    :Bars   [metric,configuration,rfc]   (median)
   .Bars.XI   :Bars   [metric,configuration,rfc]   (median)
   .Bars.XII  :Bars   [metric,configuration,rfc]   (median)

#### 4g Scatter plots by attribute

Scatter plots of location metric values and location characteristics can provide insight about the relationship between the two - i.e., does model performance have a clear relationship with any of the characteristics?

First review the attributes added to the (3rd) query above to see what the options are (or run a new query to add others).  
Let's create scatter plots of 

In [48]:
# As examples, let's create scatter plots of KGE with each of the numeric attributes

location_chars = [
    'aridity',
    'runoff_ratio',
    'baseflow_index',
    'stream_order',  
    'q_mean_cms',
    'slope_fdc',  
    'frac_urban',
    'frac_snow',
    'forest_frac'
]
df_atts[location_chars] = df_atts[location_chars].apply(pd.to_numeric)
df_atts['config_num'] = np.where(df_atts['configuration']=='nwm30_retro',1,2)

metrics = [
    'kling_gupta_efficiency_mod2',
    'pearson_correlation',                  
    'mean_absolute_relative_error',
    'relative_bias',
]
from bokeh.models import FixedTicker

scatter_layout = []
for char in location_chars:
    scatter_layout.append(
        hv.Scatter(
            df_atts, 
            kdims=[char],
            vdims=['kling_gupta_efficiency_mod2', 'relative_bias', 'primary_location_id','config_num']
        ).opts(
            width = 400, height = 300,
            #color = 'relative_bias',
            color = 'config_num',
            cmap = ['#377EB8', '#E41A1C'],
            colorbar = True,
            clim=(0.5,2.5),
            ylabel = "KGE''",
            tools=['hover'],
            ylim=(-1,1),
            size=4,
            alpha=0.8,
            colorbar_opts={
                'ticker': FixedTicker(ticks=[1,2]),
                'major_label_overrides': {
                    1: 'nwm', 
                    2: 'hbv', 
                },
                'major_label_text_align': 'left',
            },
        ))
hv.Layout(scatter_layout).cols(3)        

:Layout
   .Scatter.I    :Scatter   [aridity]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.II   :Scatter   [runoff_ratio]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.III  :Scatter   [baseflow_index]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.IV   :Scatter   [stream_order]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.V    :Scatter   [q_mean_cms]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.VI   :Scatter   [slope_fdc]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.VII  :Scatter   [frac_urban]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.VIII :Scatter   [frac_snow]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)
   .Scatter.IX   :Scatter   [forest_frac]   (kling_gupta_efficiency_mod2,relative_bias,primary_location_id,config_num)